### Database
E scooter record: https://data.cityofchicago.org/Transportation/E-Scooter-Trips/2i5w-ykuw/about_data

Divvy record: https://data.cityofchicago.org/Transportation/Divvy-Trips/fg6s-gzvg/about_data


## Get ready for analysis

### Filter data from website
I chose person_type = 'BICYCLIST' from the Traffic Crashes – People database and vehicle_type = 'MOPED OR MOTORIZED BICYCLE' from the Traffic Crashes – Vehicles database to get the crash IDs for accidents involving electric bikes, bikes and mopeds for further analysis.

Because the e‑scooter record data is too large, I will focus my analysis on the year 2023.

## Data Preprocessing

In [31]:
import pandas as pd
import numpy as np
import doctest
doctest.testmod(verbose=True)

Trying:
    time_change("04/01/2024 12:00:00 AM")
Expecting:
    1
**********************************************************************
File "__main__", line 5, in __main__.time_change
Failed example:
    time_change("04/01/2024 12:00:00 AM")
Exception raised:
    Traceback (most recent call last):
      File "C:\kelsey\Lib\doctest.py", line 1368, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest __main__.time_change[0]>", line 1, in <module>
        time_change("04/01/2024 12:00:00 AM")
      File "C:\Users\Yiting LI\AppData\Local\Temp\ipykernel_29416\1401884290.py", line 8, in time_change
        df["Start Time"] = pd.to_datetime(df["Start Time"],format="%m/%d/%Y %I:%M:%S %p")
                                          ~~^^^^^^^^^^^^^^
    TypeError: string indices must be integers, not 'str'
2 items had no tests:
    __main__
    __main__.load_multiple_csv
**********************************************************************
1 items had failure

TestResults(failed=1, attempted=1)

Load the data of scooters had crash in 2024

In [32]:
crashes_scooter = pd.read_csv('Data/Traffic_Crashes_-V-Scooter1.csv')
crashes_scooter.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1973684,9d8272483c01c20bde2d269db47d217267846e5f6126fb...,12/21/2024 02:20:00 PM,1,DRIVER,NaN,1880498,NaN,UNKNOWN,MOTORIZED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971739,81b9bd1142a056a5dd0a1e6228e3abdd2444176800fda3...,12/18/2024 05:38:00 PM,1,DRIVER,NaN,1878624,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1971226,4fc7463c094e70a614198bd5d9555768da122382e70e93...,12/17/2024 10:56:00 PM,1,DRIVER,1.0,1878140,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1967911,92dce173be505a7846ca571fcee740e5e3a79d354bf614...,12/13/2024 12:58:00 PM,1,DRIVER,NaN,1876310,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1966169,f4ba62f428523478885a2c2576a9ac4d6c8b84aded732f...,12/10/2024 12:00:00 AM,2,PARKED,NaN,1873275,NaN,CHEVROLET,BLAZER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load the details of crashes

In [33]:
crashes_details=pd.read_csv('Data/Traffic_Crashes_-_Crashes.csv')
crashes_details.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,LATITUDE,LONGITUDE
0,fde5e02103debbb95ac6b0e2391cd09fc30dbbf5a96d75...,05/02/2025 02:22:00 AM,10,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO INJURY / DRIVE AWAY,"$501 - $1,500",NOT APPLICABLE,41.865112,-87.623952
1,aa49b50898cebd8863b01a8b6e4da3cdcc3a2326803e3f...,05/01/2025 10:47:00 PM,15,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO INJURY / DRIVE AWAY,$500 OR LESS,DRIVING ON WRONG SIDE/WRONG WAY,41.764397,-87.583777
2,8c6dc50a4a5a5f4a73cdba0049c12c1630f4608014daa8...,05/01/2025 10:45:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",IMPROPER OVERTAKING/PASSING,41.869643,-87.645520
3,5a820f4840ac52672f2c15ba6246ddb6f4896191c1bad3...,05/01/2025 10:20:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO INJURY / DRIVE AWAY,"OVER $1,500",FAILING TO YIELD RIGHT-OF-WAY,41.750262,-87.663580
4,c3a270cd9bc718644ca132a4b3f356b15fd2e81fab9495...,05/01/2025 09:46:00 PM,25,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN...,41.850762,-87.706295


Find the details of crashes and vehicles with using CRASH_RECORD_ID

In [34]:
filtered_details = crashes_details[crashes_details['CRASH_RECORD_ID'].isin(crashes_scooter['CRASH_RECORD_ID'])].copy()
filtered_details.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,LATITUDE,LONGITUDE
36177,9d8272483c01c20bde2d269db47d217267846e5f6126fb...,12/21/2024 02:20:00 PM,25,CLEAR,DAYLIGHT,DRY,INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",FAILING TO YIELD RIGHT-OF-WAY,41.948914,-87.669560
37075,81b9bd1142a056a5dd0a1e6228e3abdd2444176800fda3...,12/18/2024 05:38:00 PM,15,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",NOT APPLICABLE,42.017062,-87.681798
37317,4fc7463c094e70a614198bd5d9555768da122382e70e93...,12/17/2024 10:56:00 PM,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",DISREGARDING TRAFFIC SIGNALS,41.878151,-87.633583
38815,92dce173be505a7846ca571fcee740e5e3a79d354bf614...,12/13/2024 12:58:00 PM,15,CLEAR,DAYLIGHT,DRY,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,41.968783,-87.723008
39806,f4ba62f428523478885a2c2576a9ac4d6c8b84aded732f...,12/10/2024 12:00:00 AM,30,CLEAR,UNKNOWN,DRY,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,41.793478,-87.674426


Write the method for combining multiple csv

In [35]:
import glob
import os

In [36]:
def time_change(df:pd.DataFrame) :
    """
    :param df:
    :return:
    >>> time_change("04/01/2024 12:00:00 AM")
    2
    """
    df["Start Time"] = pd.to_datetime(df["Start Time"],format="%m/%d/%Y %I:%M:%S %p")
    df["End Time"] = pd.to_datetime(df["End Time"],format="%m/%d/%Y %I:%M:%S %p")
    return df


In [37]:
def load_multiple_csv(files) -> pd.DataFrame:
    dfs=[]

    for file in files:
        df=pd.read_csv(file)
        df=time_change(df)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

combined the e-scooter record

In [38]:
scooter_records=glob.glob(os.path.join("Data/ScooterTrips_Split","*.csv"))
scooter_record=load_multiple_csv(scooter_records)

In [30]:
scooter_record.head()

,Trip ID,Start Time,End Time,Trip Distance,Trip Duration,Start Centroid Latitude,Start Centroid Longitude,End Centroid Latitude,End Centroid Longitude
0,1aa42ec4-c083-0db4-1aa4-2ec4c0830db4,2024-03-01,2024-03-01,2861,855,41.874005,-87.663518,41.901207,-87.676357
1,7fb18150-2212-43e3-b7b4-8d31e318be49,2024-03-01,2024-03-01,2871,680,41.922760,-87.699156,41.922760,-87.699156
2,4cb58624-0013-4998-b8c6-b73a25e4db31,2024-03-01,2024-03-01,1673,365,41.874005,-87.663518,41.901207,-87.676357
3,37add166-3102-4ad5-b5a4-5cf3329eab25,2024-03-01,2024-03-01,1417,322,41.944227,-87.655999,41.944227,-87.655999
4,82cb63f3-de3c-4590-b112-afe2d9cfaddd,2024-03-01,2024-03-01,12,918,41.744205,-87.656307,41.744205,-87.656307
